In [172]:
import os
import openai
import httpx

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory

import re

## 0. 事前準備

In [162]:
# APIキー設定
load_dotenv()
openai_api_key = os.environ["OPENAI_API_KEY"]

In [163]:
# モデル選択
model_list = {
    1:"gpt-3.5-turbo",
    2:"gpt-4o"
}

model_choice = 1

In [ ]:
# モデル定義
llm = ChatOpenAI(
    model=model_list[model_choice],
    temperature=0.7,
    openai_api_key=openai_api_key
)

# OutputParser
StrOutputParser = StrOutputParser()

In [165]:
# メモリ準備
memory = ConversationBufferMemory(return_messages=True)

### ユーザプロファイル

In [166]:
# ユーザプロファイル
profile = {
    "age": 30,
    "current_gyokai": "自動車",
    "current_job": "機械系エンジニア",
    "role": "リーダー",
    "experience_years": 5,

    "target_gyokai": "コンサル",
    "target_job": "データサイエンティスト"
}

### 共通プロンプト

In [167]:
# 共通ルールプロンプト
rules = f"""
# 前提条件
- タイトル：他の企業へ転職するための面接対策ロールプレイ実施
- 求職者情報：
    - 年齢：{profile["age"]}
    - 現在の業界：{profile["current_gyokai"]}業界
    - 現在の職種；{profile["current_job"]}
    - 現在の役割；{profile["role"]}
    - 経験年数：{profile["experience_years"]}
- 面接官情報：{profile["target_gyokai"]}業界、{profile["target_job"]}の経験が15年の部長で面接官
- 目的と目標：面接のロールプレイを通して依頼者の面接スキルアップ
- 評価基準：
　- リアリティのある面接ロールプレイであること
　- 依頼者の面接スキルが上がるフィードバックをすること

# 制約事項
- 面接対象者が異業界または異業種への職種なのかを考慮し、質問内容を変えること
　例：転職先特有の求められる人格を持っているかを判断するための質問をする
- 職種未経験転職であれば専門スキルレベルのみは問うレベルは低く設定すること
- 面接対象者の年齢を考慮して質問内容を変えること
　例：20代：タスクを自分ごととして実行できるかを問う
　　　30代前半：リーダとして部下のマネジメント、育成について問う
　　　30代後半以降：事業を考えたマネジメントについて問う
- ロールプレイは実際の面接官が話すような口調で実施すること
- 面接が始まったら"【質問1】〇〇を図る質問"などタイトルにあたる部分は表示せず、リアルの面接のように質問のみを表示すること
"""

### 共通関数

In [173]:
# 読点で改行する関数
def add_newlines_by_period(text):
    # 。？！（全角）で文を区切って改行を入れる
    return re.sub(r'(?<=[。？！])', '\n', text)

In [187]:
# 会話履歴保存関数
def get_history_text(memory):
    return "\n".join([
        f"{'面接官' if isinstance(m, AIMessage) else 'あなた'}：{m.content}"
        for m in memory.chat_memory.messages
    ])

## 1. Chain定義

### 1.1 自己紹介要求

In [168]:
# 定型文
intro_message = (
"""それでは、最初にあなたの自己紹介を1分（400字程度）でお願いします。
これまでのご経歴やスキルについても触れていただければと思います。"""
)

### 1.2 質問生成chain

In [169]:
# プロンプト要素
## 質問リスト
questions_list = [
    {
        "title": "転職全般に関する質問",
        "point_keys": ["定着性"],
        "content": """- 転職全般に関する質問
　- 評価ポイント："定着性"
　- 例：志望動機、転職理由、なぜこの職種なのか　など
　- 職種と業界どちらも現在と希望でことのある場合は、職種に関する質問を優先すること"""
    },
    {
        "title": "現職の取り組みに関する質問",
        "point_keys": ["課題解決力", "自走力"],
        "content": """- 現職の取り組みに関する質問
　- 評価ポイント："課題解決力"、"自走力"
　- 例：現在の仕事で課題があったとき、どう対応したか
　　　　現在の仕事で一番頑張ったこと"""
    },
    {
        "title": "職種に適正があるかを図る質問",
        "point_keys": ["自走力", "スキル"],
        "content": """- 職種に適正があるかを図る質問
　- 評価ポイント："自走力"、"スキル"
　- 例：営業であれば失敗してもへこたれずに継続する力が必要なので「業務で失敗した経験と、失敗した後何をしたか」を聞く
　　　エンジニアやデータサイエンティストであれば自己研鑽力、キャッチアップ力が必要なので「業務外で取り組んでいる学習は何か」を聞く"""
    },
    {
        "title": "職務を全うできるスキルがあるかを問う質問",
        "point_keys": ["スキル"],
        "content": """- 職務を全うできるスキルがあるかを問う質問
　- 評価ポイント："スキル"
　- 例：データアナリストであれば、「データを分析してビジネスに活用したことがあるか」を聞く"""
    }
]

## 評価ポイント
evaluation_points_list = {
    "コミュニケーション力": "質疑内容からQAの不一致がないか、結論ベースで会話ができるかを評価する",
    "定着性": "転職理由、志望動機などから長く働いてくれるかを評価する",
    "課題解決力": "課題に対して仮説思考を持って解決するための行動ができるかを評価する",
    "自走力": "仕事を自分ごととして捉えてやり切る力があるかを評価する",
    "スキル": "対象の業界、職種で活躍するために最低限必要なスキル、知識があるかを評価する"
}

In [170]:
# プロンプトテンプレート定義
question_prompt = ChatPromptTemplate.from_template("""
{rules}

# 実行指示
"前提条件"、"制約事項"を考慮して、"求職者情報"を参考に"評価ポイント"を評価できる"質問項目"に関する転職面接にありそうなリアルな質問を1つ出力すること。
また、"会話履歴"も考慮すること。

# 質問事項
{question}

# 評価ポイント
{evaluation_points}

# 会話履歴
{history}""")

# チェーン定義
create_question_chain = question_prompt | llm | StrOutputParser

## テスト

### 自己紹介要求chainテスト

In [193]:
# 面接官セリフ
print("👨‍💼 面接官：")
print(intro_message)

# ユーザ自己紹介入力
user_intro = input("🧑 あなた：")

# メモリ保存
memory.chat_memory.add_message(AIMessage(content=intro_message))
memory.chat_memory.add_message(HumanMessage(content=user_intro))

# 会話履歴格納
history_text = get_history_text(memory)
print(history_text)
# 履歴に記録されているか表示
# print("\n現在の会話メモリ：")
# for m in memory.chat_memory.messages:
#     role = "面接官" if isinstance(m, AIMessage) else "あなた"
#     print(f"{role}：{m.content}")

👨‍💼 面接官：
それでは、最初にあなたの自己紹介を1分（400字程度）でお願いします。
これまでのご経歴やスキルについても触れていただければと思います。
面接官：それでは、最初にあなたの自己紹介を1分（400字程度）でお願いします。
これまでのご経歴やスキルについても触れていただければと思います。
あなた：あいうえお


In [192]:
# メモリクリア
memory.clear()

### 質問生成chain

In [197]:
# 1問ずつ処理する場合の例（例：0番目の質問）
i = 3
selected_q = questions_list[i]

evaluation_points = "\n".join(
    [f"- {k}：{evaluation_points_list[k]}" for k in selected_q["point_keys"]]
)

question = selected_q["content"]

# チェーン実行時の引数
inputs = {
    "question": question,
    "evaluation_points": evaluation_points
}

# print(inputs["question"])
# print()
# print(inputs["evaluation_points"])

# チェーン呼び出し
output = create_question_chain.invoke({
    "rules": rules,
    "question": question,
    "evaluation_points": evaluation_points,
    "history": history_text
})

# 質問出力
formatted_output = add_newlines_by_period(output)
print("👨‍💼 面接官の質問：")
print(formatted_output)

# ユーザ回答
user_answer = input("🧑 あなた：")

# 会話メモリ保存
memory.chat_memory.add_message(AIMessage(content=output))
memory.chat_memory.add_message(HumanMessage(content=user_answer))

# 会話履歴格納
history_text = get_history_text(memory)

👨‍💼 面接官の質問：
- 「自動車業界からコンサル業界への転職にあたり、現在の業界で培った技術や知識をどのように活かす予定ですか？
」


### テンポラリ

In [198]:
print(history_text)

面接官：それでは、最初にあなたの自己紹介を1分（400字程度）でお願いします。
これまでのご経歴やスキルについても触れていただければと思います。
あなた：あいうえお
面接官：- 自動車業界からコンサル業界に転職する際、プロジェクトマネジメントやクライアント対応のスキルが求められます。過去にどのようなプロジェクトをリードし、成功に導いた経験がありますか？
あなた：たくさんあります。
面接官：- 「自動車業界からコンサル業界への転職にあたり、現在の業界で培った技術や知識をどのように活かす予定ですか？」
あなた：頑張ります。


In [184]:
print(inputs["question"])
print()
print(inputs["evaluation_points"])

- 現職の取り組みに関する質問
　- 評価ポイント："課題解決力"、"自走力"
　- 例：現在の仕事で課題があったとき、どう対応したか
　　　　現在の仕事で一番頑張ったこと

- 課題解決力：課題に対して仮説思考を持って解決するための行動ができるかを評価する
- 自走力：仕事を自分ごととして捉えてやり切る力があるかを評価する


In [108]:
prompt_text = question_prompt.format(
    rules=rules,
    question=question,
    evaluation_points=evaluation_points
)

# 出力して確認
print(prompt_text)

Human: 

# 前提条件
- タイトル：他の企業へ転職するための面接対策ロールプレイ実施
- 求職者情報：
    - 年齢：30
    - 現在の業界：自動車業界
    - 現在の職種；機械系エンジニア
    - 現在の役割；リーダー
    - 経験年数：5
- 面接官情報：コンサル業界、データサイエンティストの経験が15年の部長で面接官
- 目的と目標：面接のロールプレイを通して依頼者の面接スキルアップ
- 評価基準：
　- リアリティのある面接ロールプレイであること
　- 依頼者の面接スキルが上がるフィードバックをすること

# 制約事項
- 面接対象者が異業界または異業種への職種なのかを考慮し、質問内容を変えること
　例：転職先特有の求められる人格を持っているかを判断するための質問をする
- 職種未経験転職であれば専門スキルレベルのみは問うレベルは低く設定すること
- 面接対象者の年齢を考慮して質問内容を変えること
　例：20代：タスクを自分ごととして実行できるかを問う
　　　30代前半：リーダとして部下のマネジメント、育成について問う
　　　30代後半以降：事業を考えたマネジメントについて問う
- ロールプレイは実際の面接官が話すような口調で実施すること
- 面接が始まったら"【質問1】〇〇を図る質問"などタイトルにあたる部分は表示せず、リアルの面接のように質問のみを表示すること


# 実行指示
求職者情報を参考に評価ポイントを評価できる質問項目に関する質問を1つ出力すること。


# 質問事項
- 転職全般に関する質問
　- 評価ポイント："定着性"
　- 例：志望動機、転職理由、なぜこの職種なのか　など

# 評価ポイント
- 定着性：転職理由、志望動機などから長く働いてくれるかを評価する



In [15]:
age = input("あなたの年齢を入力してください：")
current_job = input("現在の職種を入力してください：")
target_job = input("志望している職種を入力してください：")

In [19]:
# Systemプロンプトの定義
system_prompt = f"""
あなたは{target_job}職の面接官です。
面接対象者は{age}歳で、現在の職種は{current_job}です。
転職面接のロールプレイを開始します。まず1問、面接でよく聞かれる質問を1つしてください。
"""

# 会話のやりとり
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="質問をお願いします")
]

# モデルに実行
response = llm.invoke(messages)

# 出力
print("\n👨‍💼 面接官の質問：")
print(response.content)


👨‍💼 面接官の質問：
組み込みシステムエンジニアとしての経験を活かして、データサイエンスの世界に興味を持ったきっかけは何ですか？


In [21]:
print("age:", age)
print("current_job:", current_job)
print("target_job:", target_job)

age: 30
current_job: 組み込みシステムエンジニア
target_job: データサイエンティスト


In [101]:
print(question)
print()
print(evaluation_points)

- 転職全般に関する質問
　- 評価ポイント："定着性"
　- 例：志望動機、転職理由、なぜこの職種なのか　など

- 定着性：転職理由、志望動機などから長く働いてくれるかを評価する
